In [ ]:
import numpy as np
import otmol as otm
import pandas as pd
from openbabel import pybel
#import matplotlib.pyplot as plt
#import seaborn as sns
import os
#from rdkit.Chem import MolFromPDBFile
#from experiment_utils import interactive_alignment_plot_py3dmol, copy_and_rename_pdb_files
#from experiment_utils import get_transformation_matrix, find_assignment
# %matplotlib widget

In [ ]:
# directly calculate using pybel
for ligand in ['215d_DNA', '8w4x_BGCGLC', '8w4x_BGC', '6ln3_ATP', '5bvs_EIC', '4csv_imatinib', '1ln1_DLP']:
    data_path = "../Data/extra_bio_ligands"
    df = pd.read_csv(f'arbalign_output/{ligand}_results.csv')
    mismatched_bond = []
    mismatched_bond_indices = []
    for i in range(len(df)):
        path1 = os.path.join('arbalign_output', df.iloc[i]['nameB']+'-aligned_to-'+df.iloc[i]['nameA']+'.xyz')
        path2 = os.path.join(data_path, 'extra_bio_ligands_reference', df.iloc[i]['nameA']+'.xyz')
        molA = next(pybel.readfile("xyz", path1))
        molB = next(pybel.readfile("xyz", path2))
        X_A, T_A, B_A = otm.tl.process_molecule(molA, heavy_atoms_only=True) 
        X_B, T_B, B_B = otm.tl.process_molecule(molB, heavy_atoms_only=True)
        mismatched_bond_count, indices = otm.tl.mismatched_bond_counter(B_A, B_B, np.arange(X_A.shape[0]), B_A.shape[0], B_B.shape[0], only_A_bonds=True)
        mismatched_bond.append(mismatched_bond_count/np.sum(B_A)*200)
        mismatched_bond_indices.append(indices)
    df['% mismatched bond'] = mismatched_bond
    df['mismatched bond indices'] = mismatched_bond_indices # BCI
    df.to_csv(f'arbalign_output/{ligand}_results_with_mismatched_bond.csv', index=False)

In [2]:
data_path = "../data/S1-MA-W1"
df = pd.read_csv(f'arbalign_output/S1MAW1_result.csv')
mismatched_bond = []
mismatched_bond_indices = []
for i in range(len(df)):
    path1 = os.path.join(data_path, df.iloc[i]['nameA']+'.xyz')
    path2 = os.path.join('arbalign_output', df.iloc[i]['nameB']+'-aligned_to-'+df.iloc[i]['nameA']+'.xyz')
    molA = next(pybel.readfile("xyz", path1))
    molB = next(pybel.readfile("xyz", path2))
    X_A, T_A, B_A = otm.tl.process_molecule(molA) 
    X_B, T_B, B_B = otm.tl.process_molecule(molB)
    mismatched_bond_count, indices = otm.tl.mismatched_bond_counter(B_A, B_B, np.arange(X_A.shape[0]), B_A.shape[0], B_B.shape[0], only_A_bonds=True)
    mismatched_bond.append(mismatched_bond_count/np.sum(B_A)*200)
    mismatched_bond_indices.append(indices)
df['% mismatched bond'] = mismatched_bond
df['bond indices in A that are not in B'] = mismatched_bond_indices
df.to_csv(f'arbalign_output/S1MAW1_results_with_mismatched_bond.csv', index=False)

In [16]:
df = pd.read_csv('arbalign_output/4csv_imatinib_results_with_mismatched_bond.csv')
df.iloc[50]['nameB']
df.iloc[50]['mismatched bond indices']

'[(1, 5), (2, 4), (3, 5), (3, 18), (4, 7), (6, 18), (7, 9), (7, 11), (8, 11), (8, 36), (10, 13), (10, 36), (12, 18), (12, 20), (13, 17), (16, 19), (20, 26), (21, 24), (22, 31), (23, 28), (24, 34), (25, 33), (27, 34), (28, 32), (29, 31), (30, 32)]'

In [ ]:
"""
215 DNA has a strange bug, not sure why.
for ligand in ['1ln1_DLP', '5bvs_EIC']:
    data_path = "../Data/extra_bio_ligands"
    df = pd.read_csv(f'arbalign_output/{ligand}_results.csv')
    mismatched_bond = []
    for i in range(len(df)):
        target_path = os.path.join(data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', f'boltz_results_{ligand}', df.iloc[i]['nameB']+'.xyz')
        arbalign_path = os.path.join('arbalign_output', df.iloc[i]['nameB']+'-aligned_to-'+df.iloc[i]['nameA']+'.xyz')
        mol_target = next(pybel.readfile("xyz", target_path))
        mol_arbalign = next(pybel.readfile("xyz", arbalign_path))
        X_target, _, _ = otm.tl.process_molecule(mol_target, heavy_atoms_only=True)
        X_arbalign, _, _ = otm.tl.process_molecule(mol_arbalign, heavy_atoms_only=True)
        swap, reflect = df.iloc[i]['swap'], df.iloc[i]['reflect']
        swap = list(map(int, swap.replace('(', '').replace(')', '').split(',')))
        reflect = list(map(int, reflect.replace('(', '').replace(')', '').split(',')))
        X_arbalign = X_arbalign @ (np.linalg.inv(get_transformation_matrix(swap, reflect))).T
        assignment = find_assignment(X_arbalign, X_target)
        path1 = os.path.join(data_path, 'extra_bio_ligands_reference', df.iloc[i]['nameA']+'.pdb')
        path2 = os.path.join(
                data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', f'boltz_results_{ligand}', df.iloc[i]['nameB']+'.pdb'
            )  
        X_A, T_A, B_A = otm.tl.parse_pdb_file(path1)
        X_B, T_B, B_B = otm.tl.parse_pdb_file(path2)
        mismatched_bond.append(otm.tl.mismatched_bond_counter(B_A, B_B, assignment, B_A.shape[0], B_B.shape[0], only_look_at_bonds=True)/np.sum(B_A)*200)
    df['% mismatched bond'] = mismatched_bond
    df.to_csv(f'arbalign_output/{ligand}_results_with_mismatched_bond.csv', index=False)
"""

In [ ]:
"""
# first infer the assignment using arbalign output
data_path = "../Data/extra_bio_ligands"
for ligand in ['4csv_imatinib', '6ln3_ATP']:
    df = pd.read_csv(f'arbalign_output/{ligand}_results.csv')
    mismatched_bond = []
    for i in range(51):
        if i < 50:
            target_path = os.path.join(data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', f'boltz_results_{ligand}', df.iloc[i]['nameB']+'.xyz')
        if i == 50:
            target_path = os.path.join(data_path, 'extra_bio_ligands_CSD', df.iloc[i]['nameB']+'.xyz')
        arbalign_path = os.path.join('arbalign_output', df.iloc[i]['nameB']+'-aligned_to-'+df.iloc[i]['nameA']+'.xyz')
        mol_target = next(pybel.readfile("xyz", target_path))
        mol_arbalign = next(pybel.readfile("xyz", arbalign_path))
        X_target, _, _ = otm.tl.process_molecule(mol_target, heavy_atoms_only=True)
        X_arbalign, _, _ = otm.tl.process_molecule(mol_arbalign, heavy_atoms_only=True)
        swap, reflect = df.iloc[i]['swap'], df.iloc[i]['reflect']
        swap = list(map(int, swap.replace('(', '').replace(')', '').split(',')))
        reflect = list(map(int, reflect.replace('(', '').replace(')', '').split(',')))
        X_arbalign = X_arbalign @ (np.linalg.inv(get_transformation_matrix(swap, reflect))).T
        assignment = find_assignment(X_arbalign, X_target)
        path1 = os.path.join(data_path, 'extra_bio_ligands_reference', df.iloc[i]['nameA']+'.pdb')
        if i < 50:
            path2 = os.path.join(
                data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', f'boltz_results_{ligand}', df.iloc[i]['nameB']+'.pdb'
            )  
        if i == 50:
            path2 = os.path.join(
                data_path, 'extra_bio_ligands_CSD', df.iloc[i]['nameB']+'.pdb'
            )  
        X_A, T_A, B_A = otm.tl.parse_pdb_file(path1)
        X_B, T_B, B_B = otm.tl.parse_pdb_file(path2)
        mismatched_bond.append(otm.tl.mismatched_bond_counter(B_A, B_B, assignment, B_A.shape[0], B_B.shape[0], only_look_at_bonds=True)/np.sum(B_A)*200)
    df['% mismatched bond'] = mismatched_bond
    df.to_csv(f'arbalign_output/{ligand}_results_with_mismatched_bond.csv', index=False)
"""

In [ ]:
"""
data_path = "../Data/extra_bio_ligands"
for ligand in ['8w4x_BGCGLC', '8w4x_BGC']:
    df = pd.read_csv(f'arbalign_output/{ligand}_results.csv')
    mismatched_bond = []
    for i in range(len(df)):
        target_path = os.path.join(data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', 'boltz_results_8w4x_sugars', df.iloc[i]['nameB']+'.xyz')
        arbalign_path = os.path.join('arbalign_output', df.iloc[i]['nameB']+'-aligned_to-'+df.iloc[i]['nameA']+'.xyz')
        mol_target = next(pybel.readfile("xyz", target_path))
        mol_arbalign = next(pybel.readfile("xyz", arbalign_path))
        X_target, _, _ = otm.tl.process_molecule(mol_target, heavy_atoms_only=True)
        X_arbalign, _, _ = otm.tl.process_molecule(mol_arbalign, heavy_atoms_only=True)
        swap, reflect = df.iloc[i]['swap'], df.iloc[i]['reflect']
        swap = list(map(int, swap.replace('(', '').replace(')', '').split(',')))
        reflect = list(map(int, reflect.replace('(', '').replace(')', '').split(',')))
        #X_target = X_target @ (get_transformation_matrix(swap, reflect)).T
        X_arbalign = X_arbalign @ (np.linalg.inv(get_transformation_matrix(swap, reflect))).T
        assignment = find_assignment(X_arbalign, X_target)
        path1 = os.path.join(data_path, 'extra_bio_ligands_reference', df.iloc[i]['nameA']+'.pdb')
        path2 = os.path.join(
                data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', 'boltz_results_8w4x_sugars', df.iloc[i]['nameB']+'.pdb'
            )  
        X_A, T_A, B_A = otm.tl.parse_pdb_file(path1)
        X_B, T_B, B_B = otm.tl.parse_pdb_file(path2)
        mismatched_bond.append(otm.tl.mismatched_bond_counter(B_A, B_B, assignment, B_A.shape[0], B_B.shape[0], only_look_at_bonds=True)/np.sum(B_A)*200)
    df['% mismatched bond'] = mismatched_bond
    df.to_csv(f'arbalign_output/{ligand}_results_with_mismatched_bond.csv', index=False)
"""

In [ ]:
"""
ligand = '215d_DNA'
data_path = "../Data/extra_bio_ligands"
df = pd.read_csv(f'arbalign_output/{ligand}_results.csv')
mismatched_bond = []
for i in range(len(df)):
    target_path = os.path.join(data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', 'AF3_215D_DNA_chainBC', df.iloc[i]['nameB']+'.xyz')
    arbalign_path = os.path.join('arbalign_output', df.iloc[i]['nameB']+'-aligned_to-'+df.iloc[i]['nameA']+'.xyz')
    mol_target = next(pybel.readfile("xyz", target_path))
    mol_arbalign = next(pybel.readfile("xyz", arbalign_path))
    X_target, _, _ = otm.tl.process_molecule(mol_target, heavy_atoms_only=True)
    X_arbalign, _, _ = otm.tl.process_molecule(mol_arbalign, heavy_atoms_only=True)
    swap, reflect = df.iloc[i]['swap'], df.iloc[i]['reflect']
    swap = list(map(int, swap.replace('(', '').replace(')', '').split(',')))
    reflect = list(map(int, reflect.replace('(', '').replace(')', '').split(',')))
    #X_target = X_target @ (get_transformation_matrix(swap, reflect)).T
    X_arbalign = X_arbalign @ (np.linalg.inv(get_transformation_matrix(swap, reflect))).T
    #print(X_arbalign, X_target)
    assignment = find_assignment(X_arbalign, X_target)
    path1 = os.path.join(data_path, 'extra_bio_ligands_reference', df.iloc[i]['nameA']+'.pdb')
    path2 = os.path.join(
            data_path, 'extra_bio_ligands_boltz_af3_generated_conformers', 'AF3_215D_DNA_chainBC', df.iloc[i]['nameB']+'.pdb'
        )  
    X_A, T_A, B_A = otm.tl.parse_pdb_file(path1)
    X_B, T_B, B_B = otm.tl.parse_pdb_file(path2)
    mismatched_bond.append(otm.tl.mismatched_bond_counter(B_A, B_B, assignment, B_A.shape[0], B_B.shape[0], only_look_at_bonds=True)/np.sum(B_A)*200)

df['% mismatched bond'] = mismatched_bond
df.to_csv(f'arbalign_output/{ligand}_results_with_mismatched_bond.csv', index=False)
"""